In [10]:
import numpy as np
import h5py
import time
import copy
from random import randint
import time

# def load_data(filename):
#load MNIST data
MNIST_data = h5py.File(r'C:\Users\yshcc\Desktop\hw\IE534DL\hw1\MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:])
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] )
y_test = np.int32( np.array( MNIST_data['y_test'][:,0] ) )
MNIST_data.close()

def normalization(x):
    x_mean = x.mean(axis = 0)
    x_std = x.std(axis = 0)
    x_train = (x - x_mean)/x_std
    return x
x_train = normalization(x_train)
x_test = normalization(x_test)

def convert_to_onehot(Y, C): 
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
y_train = convert_to_onehot(y_train, 10)
y_test = convert_to_onehot(y_test, 10)


C:\ADORABLE\Software\anaocnda\envs\main\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [11]:
def initial_par(n_input, n_hidden, n_output):
    np.random.seed(1)
    parameters = {}
    parameters["W1"] = np.random.randn(n_hidden, n_input) / np.sqrt(n_input)   # weight matrix
    parameters["b1"] = np.zeros(n_hidden)                 # bias vector
    parameters["W2"] = np.random.randn(n_output, n_hidden) / np.sqrt(n_hidden)
    parameters["b2"] = np.zeros(n_output)
    return parameters

In [12]:
def sigmoid(z):
    s = 1.0 / (1.0 + np.exp(-z))
    return s

In [13]:
def forward(X, params):
    cache = {}
    cache["Z1"] = np.matmul(params["W1"], X) + params["b1"]
    cache["H"] = sigmoid(cache["Z1"])
#     print("H.shape",cache["H"].shape )
    cache["Z2"] = np.matmul(params["W2"], cache["H"]) + params["b2"]
    cache["F"] = np.exp(cache["Z2"]) / np.sum(np.exp(cache["Z2"]), axis=0)
    return cache


def back_propagation(X, Y, params, cache):
    grads = {}
#     print("Y", Y)
#     print("Y.shape",Y.shape)
    dZ2 = cache["F"] - Y
    
    grads["dW2"] = np.matmul(dZ2[:,np.newaxis], cache["H"][np.newaxis,:])
    grads["db2"] = dZ2
    
    dH = np.matmul(params["W2"].T, dZ2)
    dZ1 = dH * sigmoid(cache["Z1"]) * (1 - sigmoid(cache["Z1"]))
#     print(dZ1.shape)
#     print(X.shape)
    grads["dW1"] = np.matmul(dZ1[:,np.newaxis], X[np.newaxis,:])
    grads["db1"] = dZ1

    return grads

def update_par(parameters, grads, lr):
    for l in range(1,3):
        parameters['W' + str(l)] -= lr*grads['dW'+str(l)]
        parameters['b' + str(l)] -= lr*grads['db'+str(l)]
    return parameters   

In [14]:
def model(X, Y, parameters,LR = 0.1,  num_iterations = 5, print_cost = False):
    for epochs in range(num_iterations):
            #Learning rate schedule
            if (epochs > 5):
                LR = 0.001
            if (epochs > 10):
                LR = 0.0001
            if (epochs > 15):
                LR = 0.00001
            total_correct = 0
            print(LR)
            for n in range(len(x_train)):
                n_random = randint(0,len(x_train)-1 )
                y = y_train[n_random]
                x = x_train[n_random][:]    
                # forward
                cache = forward(x, parameters)
                # backward
                grads = back_propagation(x, y, parameters, cache)
                parameters = update_par(parameters, grads, LR)

#                 W1 = parameters["W1"]
#                 b1 = parameters["b1"]
#                 W2 = parameters["W2"]
#                 b2 = parameters["b2"]
                
                prediction = np.argmax(cache["F"].T)
                yy = np.argmax(y.T)
                if (prediction == yy):
                    total_correct += 1
            print(total_correct)
            print(total_correct/np.float(len(x_train)))
    return parameters

In [15]:
layers_dims = (784, 120, 10)
parameters = initial_par(784, 120, 10)
par = model(x_train, y_train, parameters,LR = 0.1,num_iterations = 5, print_cost = False)

0.1
56129
0.9354833333333333
0.1
58224
0.9704
0.1
58693
0.9782166666666666
0.1
58937
0.9822833333333333
0.1
59125
0.9854166666666667


In [22]:
total_correct = 0
for n in range(len(x_test)):
    y = y_test[n]
    x = x_test[n][:]
    cache = forward(x, par)
    prediction = np.argmax(cache["F"].T)
    yy = np.argmax(y.T)
    if (prediction == yy):
        total_correct += 1
print(total_correct/np.float(len(x_test)))

0.9805
